In [121]:
import pandas as pd
import cv2
import numpy as np
import os 
 
#Load Dataset Path 
dataset_path = os.getcwd() + "/fer2013_New.csv"
image_size=(48,48)
dataset_path

'/home/jupyter/CNN/fer2013_New.csv'

In [122]:
data = pd.read_csv(dataset_path)
pixels = data['pixels'].tolist()
width, height = 48, 48
faces = []

In [124]:
# load Dataset 
i=1
for pixel_seq in pixels:
    face = [int(pixel) for pixel in pixel_seq.split(' ')]
    face = np.asarray(face).reshape(width, height)
    face = cv2.resize(face.astype('uint8'),image_size)
    faces.append(face.astype('float32'))    
    i=i+1
    #break
faces = np.asarray(faces)
faces = np.expand_dims(faces, -1)
emotions = pd.get_dummies(data['emotion']).as_matrix()    

#Check the Dataset shape
print(emotions.shape)
print(faces.shape)

    

(35340, 6)
(35340, 48, 48, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [125]:
#It is a standard way to pre-process images by scaling them between -1 to 1. 
# Images is scaled to [0,1] by dividing it by 255

def preprocess_input(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    if v2:
        x = x - 0.5
        x = x * 2.0
    return x

In [126]:
# split the dataset in to Train and Test in 80:20 ratio

from sklearn.model_selection import train_test_split
#faces, emotions = load_data()
faces = preprocess_input(faces)
xtrain, xtest,ytrain,ytest = train_test_split(faces, emotions,test_size=0.2,shuffle=True)

In [127]:
#Load the library
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.layers import Activation, Convolution2D, Dropout, Conv2D
from keras.layers import AveragePooling2D, BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers import Flatten
from keras.models import Model
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.layers import SeparableConv2D
from keras import layers
from keras.regularizers import l2
import pandas as pd
import cv2
import numpy as np

In [128]:
# Hyperparameter
batch_size = 32
num_epochs = 110
input_shape = (48, 48, 1)
verbose = 1
num_classes = 6
patience = 50
base_path = 'models/'
l2_regularization=0.01
 
# data generator
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)
 
# model parameters
regularization = l2(l2_regularization)
 
# base
img_input = Input(input_shape)
x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization, use_bias=False)(img_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

In [129]:
#Build the model
residual = Conv2D(128, (1, 1), strides=(2, 2),padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(128, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(128, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])
x = Conv2D(num_classes, (3, 3), padding='same')(x)
x = GlobalAveragePooling2D()(x)
output = Activation('softmax',name='predictions')(x)
 
model = Model(img_input, output)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 48, 48, 1)    0                                            
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 46, 46, 8)    72          input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_16 (BatchNo (None, 46, 46, 8)    32          conv2d_13[0][0]                  
__________________________________________________________________________________________________
activation_10 (Activation)      (None, 46, 46, 8)    0           batch_normalization_16[0][0]     
__________________________________________________________________________________________________
conv2d_14 

In [ ]:
# callbacks
base_path =os.getcwd() + "/model/"
log_file_path = base_path + '_emotion_training.log'
csv_logger = CSVLogger(log_file_path, append=False)
early_stop = EarlyStopping('val_loss', patience=patience)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1, patience=int(patience/4), verbose=1)
trained_models_path = base_path + 'model'
model_names = trained_models_path + '.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,save_best_only=True)
callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]

model.fit_generator(data_generator.flow(xtrain, ytrain,batch_size),
                        steps_per_epoch=len(xtrain) / batch_size,
                        epochs=num_epochs, verbose=1, callbacks=callbacks,
                        validation_data=(xtest,ytest))

Epoch 1/110
884/883 [==============================] - 923s 1s/step - loss: 1.7835 - acc: 0.2741 - val_loss: 1.8648 - val_acc: 0.2088

Epoch 00001: val_loss improved from inf to 1.86476, saving model to /home/jupyter/CNN/model/model.01-0.21.hdf5
Epoch 2/110
884/883 [==============================] - 914s 1s/step - loss: 1.6842 - acc: 0.3113 - val_loss: 1.7112 - val_acc: 0.2842

Epoch 00002: val_loss improved from 1.86476 to 1.71118, saving model to /home/jupyter/CNN/model/model.02-0.28.hdf5
Epoch 3/110
884/883 [==============================] - 915s 1s/step - loss: 1.6516 - acc: 0.3288 - val_loss: 2.0070 - val_acc: 0.2821

Epoch 00003: val_loss did not improve from 1.71118
Epoch 4/110
884/883 [==============================] - 909s 1s/step - loss: 1.6324 - acc: 0.3414 - val_loss: 1.6678 - val_acc: 0.3291

Epoch 00004: val_loss improved from 1.71118 to 1.66782, saving model to /home/jupyter/CNN/model/model.04-0.33.hdf5
Epoch 5/110
884/883 [==============================] - 911s 1s/step 

In [ ]:
# Evaluating the model.
model.evaluate(xtest, ytest,verbose=0)

In [ ]:
# Saving the model with .h5 extension.
# once We will save this trained model then we can reuse it on other system.
model.save_weights('CNNModel.h5')